In [1]:
# TODO: will change to use "pip install -e ." for developing the package
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent))

In [2]:
import time
import re

import pandas as pd
from collections import OrderedDict

import json
import os

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [3]:
from deck_crawler.parse_deck import parse_events_from_official
from deck_crawler.parse_deck import reassign_category
from deck_crawler.utils.translator import translate_deck

In [4]:
# flags and parameters
RUN_PARSE = False
DEBUG_PARSE = False

NUM_RESULT_PAGE = 20
NUM_EVENT_PAGE = 1000
NUM_DECK_PAGE = 2

START_DATE = "2023年02月04日"  # include
END_DATE = "2023年02月10日"  # include

TARGET_CATEGORY = "夢幻VMAX"

"""
放逐_騎拉帝納VSTAR

LTB
LTB_勾魂眼_噴火龍
LTB_蓋歐卡
LTB_洗翠黏美龍VSTAR

阿爾宙斯VSTAR
阿爾_騎拉帝納VSTAR
阿爾_白馬VMAX
阿爾_鋁鋼龍VMAX
純阿爾VSTAR

夢幻VMAX

- 密勒頓ex	37

洛奇亞VSTAR	19

沙奈朵ex 	18

起源帕路奇亞VSTAR	17

連擊武道熊師VMAX	13

洗翠索羅亞克VSTAR	10

飄香豚ex 	4

達克萊伊VSTAR	4

烈空坐VMAX	2

雷吉鐸拉戈VSTAR	1

起源帝牙盧卡VSTAR	1

索羅風速狗	1

洛托姆VSTAR	1

月石	1

化石翼龍VSTAR	1
"""

'\n放逐_騎拉帝納VSTAR\n\nLTB\nLTB_勾魂眼_噴火龍\nLTB_蓋歐卡\nLTB_洗翠黏美龍VSTAR\n\n阿爾宙斯VSTAR\n阿爾_騎拉帝納VSTAR\n阿爾_白馬VMAX\n阿爾_鋁鋼龍VMAX\n純阿爾VSTAR\n\n- 夢幻VMAX\t45\n\n密勒頓ex\t37\n\n洛奇亞VSTAR\t19\n\n沙奈朵ex \t18\n\n起源帕路奇亞VSTAR\t17\n\n連擊武道熊師VMAX\t13\n\n洗翠索羅亞克VSTAR\t10\n\n飄香豚ex \t4\n\n達克萊伊VSTAR\t4\n\n烈空坐VMAX\t2\n\n雷吉鐸拉戈VSTAR\t1\n\n起源帝牙盧卡VSTAR\t1\n\n索羅風速狗\t1\n\n洛托姆VSTAR\t1\n\n月石\t1\n\n化石翼龍VSTAR\t1\n'

In [5]:
# create some folders
EXCEL_FOLDER = "excel"
DB_FOLDER = "deck_db"
folders = [EXCEL_FOLDER, DB_FOLDER]

for folder in folders:
    if not os.path.exists(folder):
        os.makedirs(folder)

if not os.path.exists(f"{EXCEL_FOLDER}/{TARGET_CATEGORY}"):
    os.makedirs(f"{EXCEL_FOLDER}/{TARGET_CATEGORY}")

In [6]:
# loading db
decks = {}
store_file_name = "deck_db/test.json"
if not DEBUG_PARSE:
    store_file_name = "deck_db/store.json"
    if os.path.exists(store_file_name):
        with open(store_file_name, "r") as f:
            decks = json.load(f)

decks = reassign_category(decks)
print("categories in previous result:")
print(decks.keys())

store_code_list = []
for category in decks.keys():
    for d in decks[category]:
        store_code_list.append(d["deck_code"])
print("\n")
print("number of decks in the previous result:")
print(len(store_code_list))

categories in previous result:
dict_keys(['夢幻VMAX', '交錯切換機', '化石翼龍VSTAR', 'LTB', 'LTB_勾魂眼_噴火龍', '雷吉艾勒奇VMAX', '密勒頓ex', 'Other_Lost', 'LTB_洗翠黏美龍VSTAR', '達克萊伊VSTAR', '起源帕路奇亞VSTAR', '白馬蕾冠王VMAX', '放逐_騎拉帝納VSTAR', '超夢 V-UNION', '阿爾宙斯VSTAR', '光輝無極汰那', '飛天皮卡丘VMAX', '酋雷姆VMAX', '起源帝牙盧卡VSTAR', 'LTB_空之封印石_水超鋼雷', 'LTB_空之封印石_水超闘雷', 'LTB_蓋歐卡', 'LTB_空之封印石_other', 'LTB_空之封印石_超鋼雷', '阿羅拉六尾VSTAR', '阿爾_飛天皮VMAX', '連擊武道熊師VMAX', 'LTB_空之封印石_超闘', 'LTB_空之封印石_水超鋼闘', 'LTB_空之封印石_水超闘', 'LTB_空之封印石_悪水超雷', 'LTB_快龍V', 'LTB_空之封印石_水超', 'LTB_空之封印石_悪水超鋼', 'LTB_空之封印石_水超鋼闘雷', 'LTB_空之封印石_草超鋼雷', 'LTB_空之封印石_水超雷', 'LTB_空之封印石_超闘雷', 'LTB_空之封印石_草超鋼闘雷', 'LTB_空之封印石_超鋼闘雷', 'LTB_空之封印石_水草超闘雷', 'LTB_空之封印石_水超鋼', 'LTB_空之封印石_草超闘雷', 'LTB_空之封印石_水草超鋼雷', 'LTB_空之封印石_水草超鋼闘雷', 'カラマネロVMAX', 'LTB_古月鳥', '阿爾_白馬VMAX', '阿爾_騎拉帝納VSTAR', '雷吉鐸拉戈VSTAR', '洛奇亞VSTAR', '沙奈朵ex ', 'Other_騎拉帝納VSTAR', '純阿爾VSTAR', '阿爾_鋁鋼龍VMAX', '烈空坐VMAX', '黑馬蕾冠王VMAX', '幻影索羅亞克', '洛托姆VSTAR', '洗翠索羅亞克VSTAR', 'ハピナスV', '月石', 'others', '柱神', 'ヒスイ ダイケンキVSTAR', 'ムゲンダイナVMAX', '飄香豚ex ', '索羅風速狗'])

In [7]:
# parse events
t1 = time.time()
if RUN_PARSE or DEBUG_PARSE:
    parse_events_from_official(
        decks,
        store_code_list,
        result_page_limit=NUM_RESULT_PAGE,
        event_page_limit=NUM_EVENT_PAGE,
        deck_page_limit=NUM_DECK_PAGE,
    )
t2 = time.time()

print()
print(f"{t2 - t1} seconds")


7.295608520507812e-05 seconds


In [8]:
# save to json
with open(store_file_name, "w") as f:
    json.dump(decks, f, ensure_ascii=False, indent=4)

In [9]:
# # show all categories we have
# total = 0
# for k in decks.keys():
#     print(f"[{k}]:\n{len(decks[k])}\n")
#     total += len(decks[k])

# print(total)

In [10]:
# show categories we have in the period
real_start_date, real_end_date = "3000年01月01日", "1000年01月01日"
number_decks = 0
deck_cnt_list = []
for k in decks.keys():
    deck_cnt = 0
    for deck in decks[k]:
        date_exclude_jp_str = deck["date"][:-3]
        if date_exclude_jp_str >= START_DATE and date_exclude_jp_str <= END_DATE:
            deck_cnt += 1

            if date_exclude_jp_str < real_start_date:
                real_start_date = date_exclude_jp_str

            if date_exclude_jp_str > real_end_date:
                real_end_date = date_exclude_jp_str

    # print(f"{k}\t{deck_cnt}\n")
    deck_cnt_list.append((deck_cnt, k))
    if k == TARGET_CATEGORY:
        number_decks = deck_cnt

In [11]:
deck_cnt_list = sorted(deck_cnt_list, reverse=True)
for num_deck, deck_name in deck_cnt_list:
    if num_deck > 0:
        print(f"{deck_name}\t{num_deck}\n")

放逐_騎拉帝納VSTAR	159

LTB	112

交錯切換機	82

阿爾宙斯VSTAR	58

LTB_勾魂眼_噴火龍	54

雷吉艾勒奇VMAX	51

夢幻VMAX	51

密勒頓ex	48

Other_騎拉帝納VSTAR	25

阿爾_騎拉帝納VSTAR	24

洛奇亞VSTAR	22

沙奈朵ex 	22

起源帕路奇亞VSTAR	19

白馬蕾冠王VMAX	17

LTB_蓋歐卡	15

連擊武道熊師VMAX	14

超夢 V-UNION	10

洗翠索羅亞克VSTAR	10

LTB_洗翠黏美龍VSTAR	7

others	6

阿爾_白馬VMAX	5

達克萊伊VSTAR	5

Other_Lost	5

黑馬蕾冠王VMAX	4

飄香豚ex 	4

純阿爾VSTAR	4

LTB_空之封印石_水超鋼闘	4

LTB_空之封印石_other	4

飛天皮卡丘VMAX	3

光輝無極汰那	3

阿爾_飛天皮VMAX	2

阿爾_鋁鋼龍VMAX	2

起源帝牙盧卡VSTAR	2

烈空坐VMAX	2

LTB_快龍V	2

雷吉鐸拉戈VSTAR	1

酋雷姆VMAX	1

索羅風速狗	1

洛托姆VSTAR	1

月石	1

化石翼龍VSTAR	1

ヒスイ ダイケンキVSTAR	1

LTB_空之封印石_超闘	1

LTB_空之封印石_水超	1

LTB_空之封印石_悪水超雷	1

LTB_空之封印石_悪水超鋼	1



In [12]:
# number of decks for the target category
print(number_decks)
if number_decks <= 0:
    raise Exception(f"No decks for {TARGET_CATEGORY} during {START_DATE} to {END_DATE}")

51


In [13]:
# Analysis
df_list = []
common_cols = ["date", "prefecture", "num_players", "rank"]
int_cols = ["num_players", "rank"]

for card_type in ["pokemons", "tools", "supporters", "stadiums", "energies"]:
    # df init
    df = pd.DataFrame()
    for _, deck in enumerate(decks[TARGET_CATEGORY]):
        deck = translate_deck(deck)

        deck_code = deck["deck_code"]  # row id
        date_exclude_jp_str = deck["date"][:-3]
        if date_exclude_jp_str < real_start_date or date_exclude_jp_str > real_end_date:
            continue
        
        pokecard = OrderedDict()
        for col in common_cols:
            pokecard[col] = deck.get(col, "")
        pokecard.update(deck[card_type])

        if _ == 0:
            df = pd.DataFrame(pokecard, index=[deck_code])
        else:
            df = pd.concat([df, pd.DataFrame(pokecard, index=[deck_code])])
    df = df.fillna(0)

    # sort rows by date
    df = df.sort_values(by=["date"], ascending=False)

    # select cols for analysis
    col_list = list(df)
    for c in common_cols:
        col_list.remove(c)

    # calculate
    num_decks = df.shape[0]
    num_used = df[col_list].sum(axis="rows", numeric_only=True)
    num_picked = df[col_list].astype(bool).sum(axis="rows")
    avg_num_used = num_used / num_picked
    pick_rate = num_picked / num_decks

    # insert rows in df
    df.loc["avg_num_used"] = {}
    df.loc["pick_rate"] = {}
    for col in col_list:
        df.loc["avg_num_used", col] = avg_num_used[col]
        df.loc["pick_rate", col] = pick_rate[col]

    # reorder index in df, move 'avg_num_used' and 'pick_rate' to top
    num_rows = df.shape[0]
    target_rows = [num_rows - 1, num_rows - 2]
    idx = target_rows + [i for i in range(len(df)) if i not in target_rows]
    df = df.iloc[idx]

    # sort cols by pick rate
    df = df.sort_values("pick_rate", axis=1, ascending=False)
    col_list = list(df)
    for c in common_cols:
        col_list.remove(c)
    df = df[common_cols + col_list]

    # format data type
    for col in int_cols:
        df[col] = df[col].astype("Int64")

    # store
    df_list.append(df)

In [14]:
# excel writer
writer = pd.ExcelWriter(
    f"{EXCEL_FOLDER}/{TARGET_CATEGORY}/{TARGET_CATEGORY}-{real_start_date}-{real_end_date}.xlsx",
    engine="xlsxwriter",
    mode="w",
)
workbook = writer.book

for sheet_id, sheet_name in enumerate(
    ["pokemons", "tools", "supporters", "stadiums", "energies"]
):
    df_list[sheet_id].to_excel(writer, sheet_name=sheet_name, float_format="%.2f")

    # Make deck_link become a hyperlink
    # Get the xlsxwriter workbook and worksheet objects
    worksheet = writer.sheets[sheet_name]
    for i, deck_id in enumerate(df_list[sheet_id].index):
        if deck_id in ["avg_num_used", "pick_rate"]:
            continue

        # Calculate the row number
        row = i + 2

        # Write the hyperlink to the cell
        worksheet.write_url(
            f"A{row}",
            f"https://www.pokemon-card.com/deck/confirm.html/deckID/{deck_id}",
            string=f"{deck_id}",
        )

    # formatting
    header_format = workbook.add_format(
        {"bold": True, "text_wrap": True, "valign": "top"}
    )
    for col_num, value in enumerate(df_list[sheet_id].columns.values):
        worksheet.write(0, col_num + 1, value, header_format)
    worksheet.autofit()

workbook.close()
writer.save()

In [15]:
df_list[0]

,date,prefecture,num_players,rank,夢幻VMAX\nS8 040/100,メロエッタ\nS8 048/100,ゲノセクトV\nS8 069/100,夢幻VMAX\nS12a 054/172,夢幻V\nS8 039/100,夢幻V\nS8 105/100,ゲノセクトV\nS8 109/100,夢幻V\nS12a 053/172,夢幻V\nS8 106/100,ゲノセクトV\nS12a 102/172,ゲノセクトV\nS8 108/100,デオキシス\nS12a 185/172,オドリドリ\nS12a 176/172,オドリドリ\nS8 017/100,デオキシス\nS8 045/100,オドリドリ\nS12a 022/172,デオキシス\nS12a 060/172,冰砌鵝\nS8b 041/184,龍王蠍V\nS12a 085/172
pick_rate,NaN,NaN,<NA>,<NA>,0.647059,0.588235,0.372549,0.352941,0.333333,0.274510,0.274510,0.235294,0.215686,0.215686,0.176471,0.176471,0.137255,0.098039,0.098039,0.098039,0.039216,0.019608,0.019608
avg_num_used,NaN,NaN,<NA>,<NA>,2.939394,1.600000,3.947368,3.000000,3.882353,3.357143,3.857143,3.666667,3.636364,3.727273,3.777778,1.111111,1.000000,1.000000,1.400000,1.000000,1.500000,1.000000,1.000000
ypXp2p-hyeQa4-M2pyyS,2023年02月10日(金),大阪府,100,14,3.000000,0.000000,0.000000,0.000000,0.000000,4.000000,0.000000,0.000000,0.000000,0.000000,4.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
NNgNLg-75VZ8R-gnnNPL,2023年02月10日(金),大阪府,100,16,3.000000,0.000000,4.000000,0.000000,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
cDaaGx-lwaJYH-xc48c8,2023年02月10日(金),大阪府,100,15,3.000000,0.000000,0.000000,0.000000,0.000000,3.000000,0.000000,0.000000,0.000000,0.000000,4.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000
XyXppE-oGqazH-3y2pMy,2023年02月09日(木),福岡県,32,2,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000,0.000000,4.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
cx88cY-Fc9j5h-Y4a8D8,2023年02月09日(木),神奈川県,64,15,3.000000,1.000000,0.000000,0.000000,0.000000,3.000000,4.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1fvFkk-PTOxtX-FfwkFF,2023年02月08日(水),東京都,64,13,0.000000,2.000000,4.000000,3.000000,0.000000,0.000000,0.000000,4.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
888aDY-GlQwRS-Gacccc,2023年02月08日(水),京都,32,5,3.000000,1.000000,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
fkdvF5-iqkCjc-fdFkFk,2023年02月08日(水),京都,32,3,0.000000,1.000000,0.000000,3.000000,0.000000,0.000000,0.000000,3.000000,0.000000,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000


In [16]:
df_list[1]

,date,prefecture,num_players,rank,ウッウロボ,高級球,對戰VIP參加證,パワータブレット,森林封印石,講究腰帶,放逐吸塵器,巢穴球,離洞繩,朋友手冊,飛羽球,交替推車,山谷回音喇叭,交錯切換機,寶可夢交替,霧之水晶,消災手套,洗翠的沉重球,超級球,きんきゅうゼリー,健行鞋,野餐籃,捕獲香氛,ポケモンキャッチャー,なべメット,消除香水,活力頭帶,全罩防守
pick_rate,NaN,NaN,<NA>,<NA>,1.000000,1.0,1.000000,1.0,1.000000,0.980392,0.960784,0.941176,0.901961,0.764706,0.725490,0.627451,0.607843,0.294118,0.215686,0.137255,0.098039,0.098039,0.078431,0.039216,0.039216,0.019608,0.019608,0.019608,0.019608,0.019608,0.019608,0.019608
avg_num_used,NaN,NaN,<NA>,<NA>,3.392157,4.0,3.882353,4.0,2.137255,1.960000,2.387755,2.333333,1.673913,1.000000,1.756757,1.906250,1.032258,4.000000,1.454545,2.142857,1.000000,1.000000,1.750000,1.000000,3.500000,1.000000,2.000000,2.000000,1.000000,1.000000,1.000000,2.000000
ypXp2p-hyeQa4-M2pyyS,2023年02月10日(金),大阪府,100,14,4.000000,4.0,4.000000,4.0,2.000000,2.000000,4.000000,3.000000,0.000000,0.000000,0.000000,2.000000,1.000000,4.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
NNgNLg-75VZ8R-gnnNPL,2023年02月10日(金),大阪府,100,16,3.000000,4.0,3.000000,4.0,2.000000,2.000000,4.000000,3.000000,2.000000,1.000000,2.000000,0.000000,1.000000,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
cDaaGx-lwaJYH-xc48c8,2023年02月10日(金),大阪府,100,15,4.000000,4.0,4.000000,4.0,3.000000,2.000000,2.000000,2.000000,2.000000,1.000000,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
XyXppE-oGqazH-3y2pMy,2023年02月09日(木),福岡県,32,2,3.000000,4.0,4.000000,4.0,2.000000,2.000000,4.000000,2.000000,0.000000,0.000000,2.000000,2.000000,1.000000,4.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
cx88cY-Fc9j5h-Y4a8D8,2023年02月09日(木),神奈川県,64,15,4.000000,4.0,4.000000,4.0,3.000000,2.000000,2.000000,2.000000,1.000000,1.000000,0.000000,3.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1fvFkk-PTOxtX-FfwkFF,2023年02月08日(水),東京都,64,13,4.000000,4.0,4.000000,4.0,2.000000,2.000000,1.000000,2.000000,1.000000,1.000000,2.000000,2.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
888aDY-GlQwRS-Gacccc,2023年02月08日(水),京都,32,5,4.000000,4.0,4.000000,4.0,2.000000,2.000000,1.000000,3.000000,1.000000,1.000000,0.000000,3.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
fkdvF5-iqkCjc-fdFkFk,2023年02月08日(水),京都,32,3,4.000000,4.0,4.000000,4.0,3.000000,2.000000,2.000000,3.000000,3.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [17]:
df_list[2]

,date,prefecture,num_players,rank,老大的指令,裁判,杜娟,カミツレのきらめき,派帕,莎莉娜,シマボシ,牡丹,ピオニー
pick_rate,NaN,NaN,<NA>,<NA>,1.0,0.980392,0.862745,0.607843,0.176471,0.117647,0.117647,0.058824,0.019608
avg_num_used,NaN,NaN,<NA>,<NA>,2.0,2.020000,1.022727,2.064516,1.333333,1.166667,1.000000,1.000000,1.000000
ypXp2p-hyeQa4-M2pyyS,2023年02月10日(金),大阪府,100,14,2.0,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
NNgNLg-75VZ8R-gnnNPL,2023年02月10日(金),大阪府,100,16,2.0,3.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
cDaaGx-lwaJYH-xc48c8,2023年02月10日(金),大阪府,100,15,2.0,1.000000,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000
XyXppE-oGqazH-3y2pMy,2023年02月09日(木),福岡県,32,2,2.0,2.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
cx88cY-Fc9j5h-Y4a8D8,2023年02月09日(木),神奈川県,64,15,2.0,1.000000,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1fvFkk-PTOxtX-FfwkFF,2023年02月08日(水),東京都,64,13,2.0,1.000000,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000
888aDY-GlQwRS-Gacccc,2023年02月08日(水),京都,32,5,2.0,1.000000,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000
fkdvF5-iqkCjc-fdFkFk,2023年02月08日(水),京都,32,3,2.0,1.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [18]:
df_list[3]

,date,prefecture,num_players,rank,通頂雪道,放逐市,結晶洞窟,崩塌的競技場
pick_rate,NaN,NaN,<NA>,<NA>,0.921569,0.666667,0.176471,0.137255
avg_num_used,NaN,NaN,<NA>,<NA>,1.978723,1.529412,1.111111,1.000000
ypXp2p-hyeQa4-M2pyyS,2023年02月10日(金),大阪府,100,14,3.000000,1.000000,0.000000,0.000000
NNgNLg-75VZ8R-gnnNPL,2023年02月10日(金),大阪府,100,16,3.000000,0.000000,0.000000,0.000000
cDaaGx-lwaJYH-xc48c8,2023年02月10日(金),大阪府,100,15,1.000000,1.000000,0.000000,1.000000
XyXppE-oGqazH-3y2pMy,2023年02月09日(木),福岡県,32,2,3.000000,0.000000,0.000000,0.000000
cx88cY-Fc9j5h-Y4a8D8,2023年02月09日(木),神奈川県,64,15,1.000000,1.000000,1.000000,0.000000
1fvFkk-PTOxtX-FfwkFF,2023年02月08日(水),東京都,64,13,1.000000,1.000000,1.000000,0.000000
888aDY-GlQwRS-Gacccc,2023年02月08日(水),京都,32,5,1.000000,1.000000,1.000000,0.000000
fkdvF5-iqkCjc-fdFkFk,2023年02月08日(水),京都,32,3,0.000000,2.000000,0.000000,0.000000


In [19]:
df_list[4]

,date,prefecture,num_players,rank,雙重渦輪能量,フュージョンエネルギー,基本超能量
pick_rate,NaN,NaN,<NA>,<NA>,1.000000,0.607843,0.058824
avg_num_used,NaN,NaN,<NA>,<NA>,3.411765,3.967742,1.000000
ypXp2p-hyeQa4-M2pyyS,2023年02月10日(金),大阪府,100,14,4.000000,0.000000,0.000000
NNgNLg-75VZ8R-gnnNPL,2023年02月10日(金),大阪府,100,16,4.000000,0.000000,0.000000
cDaaGx-lwaJYH-xc48c8,2023年02月10日(金),大阪府,100,15,4.000000,3.000000,0.000000
XyXppE-oGqazH-3y2pMy,2023年02月09日(木),福岡県,32,2,4.000000,0.000000,0.000000
cx88cY-Fc9j5h-Y4a8D8,2023年02月09日(木),神奈川県,64,15,3.000000,4.000000,0.000000
1fvFkk-PTOxtX-FfwkFF,2023年02月08日(水),東京都,64,13,3.000000,4.000000,0.000000
888aDY-GlQwRS-Gacccc,2023年02月08日(水),京都,32,5,3.000000,4.000000,0.000000
fkdvF5-iqkCjc-fdFkFk,2023年02月08日(水),京都,32,3,3.000000,4.000000,0.000000


In [20]:
len(decks["others"])

90